# notebook to run a simple glm from the budapest movie

In [1]:
import numpy as np
import pandas as pd
import nibabel as nb
import hrf_tools
import hcp_utils as hcp
from analysis import plot_results
from os import walk
from nilearn.image import load_img
import os
import gc
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp, norm
from scipy.stats import zscore
# subject_flist = list(walk(clean_path))[0][2:][0]
#all_=[]
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.glm.contrasts import compute_contrast
from nilearn.glm.first_level import run_glm


/opt/miniconda-latest/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
def load_feature(feature):
    speech_list = []
    for r in np.arange(1,6):
    #     flow = np.load(f'../sourcedata/data/budapest/features/budapest{r}_optic_flow_10hz.npy')
    #     flow_list.append(flow)
        data = pd.read_csv(f'../sourcedata/data/budapest/features/budapest{r}_{feature}.tsv', sep='\t')
        #print(data)
        speech = np.array(data['value'])
        speech = np.expand_dims(speech, axis=1).astype(float)
        speech=hrf_tools.apply_optimal_hrf_10hz(speech,10)
        speech = hrf_tools.resample_1hz(speech)
        speech_list.append(speech)
    return(speech_list)

## plotting

In [3]:
import numpy as np
import nibabel as nb
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from nilearn_plotting_custom import plot_surf
from PIL import Image
from PIL import ImageDraw
import npp
import hcp_utils as hcp
from hcp_tools import load_flatmaps_59k
from hcp_tools import load_meshes
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import resample
from sklearn.preprocessing import StandardScaler
sns.set("paper", "white")
#%matplotlib inline
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'

def plot_results(scores,score_type,data_type,vertex_info,subject,feature,dataset,title):
    '''Inputs:
        scores = data to plot
        score_type = r2, r, p, z, d, raw
        data_type = 32k (3T) or 59k (7T)
        vertex info = None or the vertex info if it is 59k data beacuse hcp_utils doesnt by default
        subject = eg 100610 subject id for file naming
        feature = eg as_scores plotted feature for file naming
        dataset = eg merlin or HCP_7T which dataset?
        title = 
    '''
    scratch_dir = '../tmp'
#     scratch_dir = '/scratch/scratch/Fri/jsmentch/tmp'
#     if not os.path.exists(scratch_dir):
#         os.mkdir(scratch_dir)
    if score_type == 'r2':
        v=[0,0.5]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'r':
        v=[0,1]
        threshold=None
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'p':
        v=[0,0.05]
        symmetric_cmap=False
        cmap='inferno'
    if score_type == 'z':
        v=[-15,15]
        threshold=3.3
        symmetric_cmap=True
        cmap='cold_hot'
    if score_type == 'd':
        v=[0,10]
        threshold=3
        symmetric_cmap=True
        cmap='inferno'
    if score_type == 'raw':
        v=[-20,40]
        threshold=1
        symmetric_cmap=True
        cmap='cold_hot'
    save_dir=f'../outputs/figures/{dataset}/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)    
    if data_type == '59k':
        flatmeshes=load_flatmaps_59k() #load flatmaps
        surf_path_msm = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/T1w/fsaverage_LR59k/100610.L.inflated_1.6mm_MSMAll.59k_fs_LR.surf.gii'
        mesh59k_msm = load_meshes(example_filename=surf_path_msm) #load other meshes
        # get data from results in plotting format
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0, vertex_info=vertex_info)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0, vertex_info=vertex_info)
        # sulcal depth paths
        sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
        sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
        # params for view to plot
        params = [('flat_L',score_cortex_dataL,flatmeshes.flat_left,sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,flatmeshes.flat_right,sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,mesh59k_msm.very_inflated_left,sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,mesh59k_msm.very_inflated_right,sulc_right,'right'),\
        ]
    elif data_type == '32k':
        score_cortex_dataL = hcp.left_cortex_data(scores, fill=0)
        score_cortex_dataR = hcp.right_cortex_data(scores, fill=0)
    #     # sulcal depth paths
    #     sulc_left = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.L.sulc.59k_fs_LR.shape.gii'
    #     sulc_right = '../sourcedata/data/human-connectome-project-openaccess/HCP1200/100610/MNINonLinear/fsaverage_LR59k/100610.R.sulc.59k_fs_LR.shape.gii'
    #     # params for view to plot
        params = [('flat_L',score_cortex_dataL,hcp.mesh.flat_left,hcp.mesh.sulc_left,'left'),\
         ('flat_R',score_cortex_dataR,hcp.mesh.flat_right,hcp.mesh.sulc_right,'right'),\
         ('vinf_L',score_cortex_dataL,hcp.mesh.very_inflated_left,hcp.mesh.sulc_left,'left'),\
         ('vinf_R',score_cortex_dataR,hcp.mesh.very_inflated_right,hcp.mesh.sulc_right,'right'),\
        ]
    # plot each hemi and mesh, save to outputs dir
    for name, data, mesh, sulc, hemi in params:
        #figure, axes = plt.subplots(subplot_kw=dict(projection="3d"), figsize=(6,4))
        plot_surf(mesh,\
                data, \
                  cmap=cmap,symmetric_cmap=symmetric_cmap, avg_method='median',#figure=fig,\
                bg_map=sulc, colorbar=True, vmin=v[0], vmax=v[1], threshold=threshold, hemi=hemi, \
#                data_alpha=np.where(data>0,1,0),\
                data_alpha=np.ones(data.shape),\
                data_remove=np.zeros(data.shape),output_file=f'{scratch_dir}/{name}.png')
#combine saved maps into one with PIL
#     if notebook==True:
    area = (75, 140, 635, 560) #area to crop from each image
#     else:
#         area = (105, 190, 880, 780)
        
    img = Image.open(f'{scratch_dir}/flat_L.png',mode='r')
    img = img.resize((770,720))
    cropped = img.crop(area)
    fL=cropped.transpose(Image.FLIP_LEFT_RIGHT)
    w,h = img.size
    c_area = (690, 0, w-10, h) # area of colorbar to crop
    cbar = img.crop(c_area)

    img = Image.open(f'{scratch_dir}/flat_R.png',mode='r')
    img = img.resize((770,720))
    fR = img.crop(area)

    img = Image.open(f'{scratch_dir}/vinf_L.png',mode='r')
    img = img.resize((770,720))
    iL = img.crop(area)
    #iL=cropped.transpose(Image.FLIP_LEFT_RIGHT)

    img = Image.open(f'{scratch_dir}/vinf_R.png',mode='r')
    img = img.resize((770,720))
    iR = img.crop(area)

    w,h=iR.size

    new_im = Image.new('RGB', ( (w*2)+70 , h*2) ,(255, 255, 255, 1))
    new_im.paste(fL,(0,h))
    new_im.paste(fR,(w,h))
    new_im.paste(iL,(0,0))
    new_im.paste(iR,(w,0))
    new_im.paste(cbar,(w*2,int(round(h/4))))

    w,h=new_im.size

    draw = ImageDraw.Draw(new_im)
    draw.text((0,0),f"{title}_{subject}_{feature}_{score_type}",(0,0,0))

    new_im.save(f'{save_dir}{title}_{subject}_{feature}_{score_type}.png')
#     os.remove(f'{scratch_dir}/flat_L.png')
#     os.remove(f'{scratch_dir}/flat_R.png')
#     os.remove(f'{scratch_dir}/vinf_L.png')
#     os.remove(f'{scratch_dir}/vinf_R.png')

## run for speech

In [5]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='speech'

clean_path = '/om/scratch/Thu/jsmentch/budapest_smoothed/cleaned/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))
# sub_list



speech_list = load_feature(feature)


second_level_input=[]
stat_maps=[]
for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = speech_list[r]
        img = load_img(f'{clean_path}sub-{sub}_task-movie_run-{r+1}_space-fsLR_den-91k_bold_deconfound.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X=np.copy(Xi)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            X=np.vstack((X,Xi))
            Y=np.vstack((Y,Yi))
    print(X.shape)
    print(Y.shape)

    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,0]).T, \
                                contrast_type='t')
    
    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
#     savepath = f'../outputs/glm/budapest/speech/'
#     os.makedirs(savepath, exist_ok=True)
#     np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')
vertex_info = hcp.get_HCP_vertex_info(img)

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

working on sub sid000142, 1 / 26
(2952, 1)
(2952, 91282)


/om2/vast/gablab/jsmentch/projects/nat_img/code/nilearn_plotting_custom.py:184: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  axes = Axes3D(figure, rect=[0, 0, 1, 1],


[0.17242915 0.17381311 0.17432143 ...        nan        nan        nan]
[0.17455096 0.17501344 0.17926516 ...        nan        nan        nan]
[0.17242915 0.17381311 0.17432143 ...        nan        nan        nan]
[0.17455096 0.17501344 0.17926516 ...        nan        nan        nan]
working on sub sid000055, 2 / 26
(2952, 1)
(2952, 91282)
[0.22327107 0.22339425 0.22439982 ...        nan        nan        nan]
[0.19198833 0.19434754 0.1944401  ...        nan        nan        nan]
[0.22327107 0.22339425 0.22439982 ...        nan        nan        nan]
[0.19198833 0.19434754 0.1944401  ...        nan        nan        nan]
working on sub sid000535, 3 / 26
(2952, 1)
(2952, 91282)
[0.26508755 0.2651953  0.26546646 ...        nan        nan        nan]
[0.24543902 0.24672358 0.24762108 ...        nan        nan        nan]
[0.26508755 0.2651953  0.26546646 ...        nan        nan        nan]
[0.24543902 0.24672358 0.24762108 ...        nan        nan        nan]
working on sub sid0000

## Now run for as_speech

In [6]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='as_speech'

clean_path = '/om/scratch/Thu/jsmentch/budapest_smoothed/cleaned/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))
# sub_list



speech_list = load_feature(feature)


second_level_input=[]
stat_maps=[]
for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)+1}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = speech_list[r]
        img = load_img(f'{clean_path}sub-{sub}_task-movie_run-{r+1}_space-fsLR_den-91k_bold_deconfound.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X=np.copy(Xi)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            X=np.vstack((X,Xi))
            Y=np.vstack((Y,Yi))
    print(X.shape)
    print(Y.shape)

    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,0]).T, \
                                contrast_type='t')
    
    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
#     savepath = f'../outputs/glm/budapest/speech/'
#     os.makedirs(savepath, exist_ok=True)
#     np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')
vertex_info = hcp.get_HCP_vertex_info(img)

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

working on sub sid000142, 1 / 26
(2952, 1)
(2952, 91282)
[0.22320134 0.22392616 0.22468375 ...        nan        nan        nan]
[0.18745369 0.18962999 0.19017379 ...        nan        nan        nan]
[0.22320134 0.22392616 0.22468375 ...        nan        nan        nan]
[0.18745369 0.18962999 0.19017379 ...        nan        nan        nan]
working on sub sid000055, 2 / 26
(2952, 1)
(2952, 91282)
[0.24168833 0.24186546 0.24294554 ...        nan        nan        nan]
[0.18965651 0.19056752 0.191019   ...        nan        nan        nan]
[0.24168833 0.24186546 0.24294554 ...        nan        nan        nan]
[0.18965651 0.19056752 0.191019   ...        nan        nan        nan]


PermissionError: [Errno 13] Permission denied: '../outputs/figures/budapest/partALL_GLM_effect_z_sid000055_as_speech_z.png'

## Now run for any_faces

In [ ]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='any_faces'

clean_path = '/om/scratch/Thu/jsmentch/budapest_smoothed/cleaned/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))
# sub_list



speech_list = load_feature(feature)


second_level_input=[]
stat_maps=[]
for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)+1}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = speech_list[r]
        img = load_img(f'{clean_path}sub-{sub}_task-movie_run-{r+1}_space-fsLR_den-91k_bold_deconfound.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X=np.copy(Xi)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            X=np.vstack((X,Xi))
            Y=np.vstack((Y,Yi))
    print(X.shape)
    print(Y.shape)

    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,0]).T, \
                                contrast_type='t')
    
    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
#     savepath = f'../outputs/glm/budapest/speech/'
#     os.makedirs(savepath, exist_ok=True)
#     np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')
vertex_info = hcp.get_HCP_vertex_info(img)

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

## CONTRAST: any_faces > home

In [ ]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='any_faces_vs_home'

clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))

face_list = load_feature('any_faces')
home_list = load_feature('home')

second_level_input=[]
stat_maps=[]

for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)+1}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = face_list[r]
        Xii = home_list[r]
        img = load_img(f'{clean_path}sub-{sub}_run{r+1}_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X1=np.copy(Xi)
            X2=np.copy(Xii)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            Xii=Xii[:-20,:]
            X2=np.vstack((X2,Xii))
            X1=np.vstack((X1,Xi))
            Y=np.vstack((Y,Yi))
    print(X1.shape)
    print(X2.shape)
    print(Y.shape)
    X = np.hstack((X1,X2))
    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,-1,0]).T, \
                                contrast_type='t')
    
#    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
    savepath = f'../outputs/glm/budapest/{feature}/'
    os.makedirs(savepath, exist_ok=True)
    np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# # z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')
vertex_info = hcp.get_HCP_vertex_info(img)

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

In [23]:
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

[0.34819226 0.34906762 0.34963217 ...        nan        nan        nan]
[0.33739895 0.33779633 0.33821753 ...        nan        nan        nan]
[0.34819226 0.34906762 0.34963217 ...        nan        nan        nan]
[0.33739895 0.33779633 0.33821753 ...        nan        nan        nan]


In [5]:
vertex_info = hcp.get_HCP_vertex_info(img)


## any_faces_nospeech

In [ ]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='any_faces_nospeech'

clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))

face_list = load_feature('any_faces')
speech_list = load_feature('speech')

second_level_input=[]
stat_maps=[]

for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)+1}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = face_list[r]
        Xii = speech_list[r]
        img = load_img(f'{clean_path}sub-{sub}_run{r+1}_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X1=np.copy(Xi)
            X2=np.copy(Xii)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            Xii=Xii[:-20,:]
            X2=np.vstack((X2,Xii))
            X1=np.vstack((X1,Xi))
            Y=np.vstack((Y,Yi))
    print(X1.shape)
    print(X2.shape)
    print(Y.shape)
    X = np.hstack((X1,X2))
    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,0,0]).T, \
                                contrast_type='t')
    
#    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
    savepath = f'../outputs/glm/budapest/{feature}/'
    os.makedirs(savepath, exist_ok=True)
    np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# # z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

In [8]:
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

[0.3510494  0.35145365 0.352387   ...        nan        nan        nan]
[0.35024786 0.35100703 0.35358208 ...        nan        nan        nan]
[0.3510494  0.35145365 0.352387   ...        nan        nan        nan]
[0.35024786 0.35100703 0.35358208 ...        nan        nan        nan]


## as_music

In [10]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='as_music'

clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))
speech_list = load_feature(feature)

second_level_input=[]
stat_maps=[]

for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)+1}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = speech_list[r]
        img = load_img(f'{clean_path}sub-{sub}_run{r+1}_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X=np.copy(Xi)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            X=np.vstack((X,Xi))
            Y=np.vstack((Y,Yi))
    print(X.shape)
    print(Y.shape)

    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,0]).T, \
                                contrast_type='t')
    
#    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
    savepath = f'../outputs/glm/budapest/{feature}/'
    os.makedirs(savepath, exist_ok=True)
    np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# # z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')
vertex_info = hcp.get_HCP_vertex_info(img)

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

working on sub sid000499, 1 / 26
(2952, 1)
(2952, 91282)
[0.32861567 0.32883001 0.32928694 ...        nan        nan        nan]
[0.31450095 0.31470865 0.31531599 ...        nan        nan        nan]
[0.32861567 0.32883001 0.32928694 ...        nan        nan        nan]
[0.31450095 0.31470865 0.31531599 ...        nan        nan        nan]
working on sub sid000120, 2 / 26
(2952, 1)
(2952, 91282)
[0.39618525 0.39666593 0.39674513 ...        nan        nan        nan]
[0.37560482 0.37565443 0.37575465 ...        nan        nan        nan]
[0.39618525 0.39666593 0.39674513 ...        nan        nan        nan]
[0.37560482 0.37565443 0.37575465 ...        nan        nan        nan]
working on sub sid000050, 3 / 26
(2952, 1)
(2952, 91282)
[0.37198408 0.37401857 0.37443519 ...        nan        nan        nan]
[0.36749002 0.36804057 0.3681963  ...        nan        nan        nan]
[0.37198408 0.37401857 0.37443519 ...        nan        nan        nan]
[0.36749002 0.36804057 0.3681963  ...

In [12]:
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

[0.30849374 0.30906387 0.31848738 ...        nan        nan        nan]
[0.3315835  0.33471202 0.33537727 ...        nan        nan        nan]
[0.30849374 0.30906387 0.31848738 ...        nan        nan        nan]
[0.3315835  0.33471202 0.33537727 ...        nan        nan        nan]


## as_music_nospeech

In [ ]:
# clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
## get list of subjects
feature='as_music_nospeech'

clean_path = '/om/scratch/Mon/jsmentch/cleaned/smoothed/'
subject_flist = list(walk(clean_path))[0][2:][0]
sub_list=[]
for s in subject_flist:
    sub = s[4:13]
    sub_list.append(sub)
sub_list=list(set(sub_list))

face_list = load_feature('as_music')
speech_list = load_feature('speech')

second_level_input=[]
stat_maps=[]

for i,sub in enumerate(sub_list):
    subject = sub[0:13]
    #if not os.path.isfile(f'../outputs/figures/budapest/part{run}_GLM_{sub}_optic_flow_z.png'):    
    print(f'working on sub {sub}, {i+1} / {len(sub_list)+1}')
    subject=sub
    dataset='budapest'
    title=f'allruns_GLM'
    for r in np.arange(5):
        Xi = face_list[r]
        Xii = speech_list[r]
        img = load_img(f'{clean_path}sub-{sub}_run{r+1}_clean_smooth_space-fsLR_den-91k_bold.dtseries.nii')
        Yi = img.get_fdata()
        #X = np.hstack((X,X_n))
        if r == 0:
            Y=np.copy(Yi[10:-10,:])
            X1=np.copy(Xi)
            X2=np.copy(Xii)
        else:
            Yi=Yi[10:-10,:]
            Xi=Xi[:-20,:]
            Xii=Xii[:-20,:]
            X2=np.vstack((X2,Xii))
            X1=np.vstack((X1,Xi))
            Y=np.vstack((Y,Yi))
    print(X1.shape)
    print(X2.shape)
    print(Y.shape)
    X = np.hstack((X1,X2))
    n_scans = Y.shape[0]
    frame_times= np.arange(n_scans)


    design_matrix = make_first_level_design_matrix(frame_times, None,
                              add_regs=X, hrf_model=None, drift_model=None)

#     from nilearn.plotting import plot_design_matrix
#     plot_design_matrix(design_matrix)

    labels,results = run_glm(Y,design_matrix.values)

#     contrast = compute_contrast(labels=labels, \
#                                 regression_result=results, \
#                                 con_val=np.array([1,0]).T, \
#                                 contrast_type='t')
    
    
    contrast = compute_contrast(labels=labels, \
                                regression_result=results, \
                                con_val=np.array([1,0,0]).T, \
                                contrast_type='t')
    
#    vertex_info = hcp.get_HCP_vertex_info(img)
#     plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,title)
    second_level_input.append(contrast.effect_size())
    stat_maps.append(contrast.stat())
    savepath = f'../outputs/glm/budapest/{feature}/'
    os.makedirs(savepath, exist_ok=True)
    np.save(f'{savepath}{sub}_stat',contrast.stat())
#     contrast.stat()
#     more_smooth_anat_img.to_filename('more_smooth_anat_img.nii.gz')
    plot_results(contrast.z_score(),'z','32k',vertex_info,subject,feature,dataset,'partALL_GLM_effect_z')
    gc.collect()
second_level_input = np.array(second_level_input)
savepath = f'../outputs/glm/budapest/{feature}/'
os.makedirs(savepath, exist_ok=True)
np.save(f'{savepath}{feature}_second_level_input',second_level_input)
np.save(f'{savepath}{feature}_stat_maps',stat_maps)

t, pval = ttest_1samp(np.nan_to_num(second_level_input), 0)
# # z_val = norm.isf(pval)
z_val = zscore(t,nan_policy='omit')

    

#np.save(f'{savepath}{feature}_zstat',z_val)
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

In [16]:
plot_results(z_val,'z','32k',vertex_info,"all",feature,dataset,'partALL_GLM_effect_z')

[0.37939429 0.38154188 0.38266425 ...        nan        nan        nan]
[0.34086334 0.34229409 0.34296669 ...        nan        nan        nan]
[0.37939429 0.38154188 0.38266425 ...        nan        nan        nan]
[0.34086334 0.34229409 0.34296669 ...        nan        nan        nan]
